In [1]:
#@title Check GPU
#@markdown Run this to connect to a Colab Instance, and see what GPU Google gave you.

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print(gpu[1])
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

Tesla P100-PCIE-16GB
The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.
Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!git clone https://github.com/junyanz/interactive-deep-colorization ideepcolor

Cloning into 'ideepcolor'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 335 (delta 3), reused 8 (delta 2), pack-reused 326
Receiving objects: 100% (335/335), 12.33 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [4]:
%cd ideepcolor

/content/ideepcolor


In [5]:
!bash ./models/fetch_models.sh

--2020-03-19 19:14:40--  http://colorization.eecs.berkeley.edu/siggraph/models/model.caffemodel
Resolving colorization.eecs.berkeley.edu (colorization.eecs.berkeley.edu)... 128.32.189.73
Connecting to colorization.eecs.berkeley.edu (colorization.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 194117235 (185M)
Saving to: ‘./models/reference_model/model.caffemodel’

./models/reference_ 100%[===================>] 185.12M  21.5MB/s    in 9.6s    

2020-03-19 19:14:50 (19.3 MB/s) - ‘./models/reference_model/model.caffemodel’ saved [194117235/194117235]

--2020-03-19 19:14:51--  http://colorization.eecs.berkeley.edu/siggraph/models/global_model.caffemodel
Resolving colorization.eecs.berkeley.edu (colorization.eecs.berkeley.edu)... 128.32.189.73
Connecting to colorization.eecs.berkeley.edu (colorization.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140028664 (134M)
Saving to: 

In [6]:
# !apt install -y caffe-cpu
!apt install -y caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  python3-scipy python3-simplegeneric python3-six python3-skimage
  python3-skimage

In [7]:
! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/ideepcolor/caffe_files"

! echo $PYTHONPATH

/tensorflow-1.15.0/python3.6:/env/python
/tensorflow-1.15.0/python3.6:/env/python:/content/ideepcolor/caffe_files


In [0]:
#sometimes this is not enough and running will fail
#in that case just move everything from `caffe_files` to parent

In [0]:
from os import listdir
from os.path import isfile, join
mypath='caffe_files'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
  !cp {'./caffe_files/'+f} {'./'+f}

In [0]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from data import colorize_image as CI
from skimage import color
from data import lab_gamut as lab

%matplotlib inline

In [0]:
base_path='//content/drive/My Drive/kontext/deold'

In [12]:
!mkdir {'"'+base_path+'/4"'}
!mkdir {'"'+base_path+'/4/colorized_auto"'}
!mkdir {'"'+base_path+'/4/colorized_ref"'}

mkdir: cannot create directory ‘//content/drive/My Drive/kontext/deold/4’: File exists
mkdir: cannot create directory ‘//content/drive/My Drive/kontext/deold/4/colorized_auto’: File exists
mkdir: cannot create directory ‘//content/drive/My Drive/kontext/deold/4/colorized_ref’: File exists


In [0]:
import caffe
import skimage

In [0]:
def load_image(filename, color=True):
    """
    Load an image converting from grayscale or alpha as needed.
    Parameters
    ----------
    filename : string
    color : boolean
        flag for color format. True (default) loads as RGB while False
        loads as intensity (if image is already grayscale).
    Returns
    -------
    image : an image with type np.float32 in range [0, 1]
        of size (H x W x 3) in RGB or
        of size (H x W x 1) in grayscale.
    """
    img = skimage.img_as_float(skimage.io.imread(filename)).astype(np.float32)
    if img.ndim == 2:
        img = img[:, :, np.newaxis]
        if color:
            img = np.tile(img, (1, 1, 3))
    elif img.shape[2] == 4:
        img = img[:, :, :3]
    return img

In [0]:
# Colorization with reference global histogram
def get_global_histogram(ref_path):
    ref_img_fullres = load_image(ref_path)
    img_glob_dist = (255*caffe.io.resize_image(ref_img_fullres,(Xd,Xd))).astype('uint8') # load image
    gt_glob_net.blobs['img_bgr'].data[...] = img_glob_dist[:,:,::-1].transpose((2,0,1)) # put into 
    gt_glob_net.forward();
    glob_dist_in = gt_glob_net.blobs['gt_glob_ab_313_drop'].data[0,:-1,0,0].copy()
    return (glob_dist_in,ref_img_fullres)

In [0]:
from os import listdir
from os.path import isfile, join
mypath=base_path+'/2/goodscenes'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [17]:
for ref in ['szf2']:
  # color histogram to use
  ref_path = ref+'.png'
  for img_path in onlyfiles:
    if ref in img_path: 
      print(img_path)   
      gpu_id = 0 # gpu to use
      Xd = 256

      # Colorization network
      cid = CI.ColorizeImageCaffeGlobDist(Xd)
      cid.prep_net(gpu_id,prototxt_path='./models/global_model/deploy_nodist.prototxt',\
                  caffemodel_path='./models/global_model/global_model.caffemodel')

      # Global distribution network - extracts global color statistics from an image
      gt_glob_net = caffe.Net('./models/global_model/global_stats.prototxt',\
                            './models/global_model/dummy.caffemodel', caffe.TEST)

      # Load image
      cid.load_image(base_path+'/2/goodscenes/'+img_path)

      # Dummy variables
      input_ab = np.zeros((2,Xd,Xd))
      input_mask = np.zeros((1,Xd,Xd))

      # Colorization without global histogram
      img_pred = cid.net_forward(input_ab,input_mask);
      img_pred_auto_fullres = cid.get_img_fullres()

      # Gray image
      img_gray_fullres = cid.get_img_gray_fullres()

      # Plot result
      plt.figure(figsize=(10,4))
      plt.imshow(np.concatenate((img_gray_fullres,img_pred_auto_fullres),axis=1));
      plt.imsave(base_path+'/4/'+'colorized_auto/'+img_path, img_pred_auto_fullres)
      plt.title('Grayscale image / Automatic colorization (no reference histogram)')
      plt.axis('off');

      (glob_dist_ref,ref_img_fullres) = get_global_histogram(base_path+'/2/goodscenes/'+ref_path)
      img_pred = cid.net_forward(input_ab,input_mask,glob_dist_ref);
      img_pred_withref_fullres = cid.get_img_fullres()

      plt.figure(figsize=(10,4))
      plt.imshow(img_gray_fullres);
      plt.title('Grayscale image')
      plt.axis('off');
      plt.figure(figsize=(10,4))
      plt.imshow(img_pred_withref_fullres);
      plt.imsave(base_path+'/4/'+'colorized_ref/'+img_path, img_pred_withref_fullres)
      plt.title('Resulting colorization with reference histogram')
      plt.axis('off');

      plt.figure(figsize=(4,4))
      plt.imshow(ref_img_fullres);
      plt.title('Reference image');
      plt.axis('off');

Output hidden; open in https://colab.research.google.com to view.